In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import util
import mylearn
import time
np.random.seed(1)
data = pd.read_csv('train.csv') #pre-downloaded housing data
House = data.copy()

In [2]:
def create_classes(dat, col = 'SalePrice'):#creates a definition that creates a class definition
    val = dat[col]
    if val < 100000: return 0
    elif val <150000: return 1
    elif val <200000: return 2
    elif val <250000: return 3
    else: return 4

We will use our best 3 accuracy rates for Naive Bayes, which is MPP (Case 2), BPNN, and and SVM

In [3]:
def mean_normal(df):
    return (df-df.mean())/df.std()

In [4]:
Housedat = pd.get_dummies(data)

In [5]:
Housedat['SaleClass'] = Housedat.apply(create_classes, axis = 1)

In [6]:
Housedat.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SaleClass
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,0,1,0,0,0,0,1,0,3
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,0,0,1,0,0,0,0,1,0,2
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,0,1,0,0,0,0,1,0,3
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,0,1,1,0,0,0,0,0,1
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,0,0,1,0,0,0,0,1,0,4


In [7]:
targets = Housedat['SaleClass']

In [8]:
Housedat.drop(['Id', 'SaleClass', 'SalePrice'], axis = 1, inplace = True)

In [9]:
Housedat['HouseAge']=2010 - Housedat['YearBuilt']
Housedat.drop('YearBuilt', axis =1, inplace = True)
Housedat['RemodAge']=2010 - Housedat['YearRemodAdd']
Housedat.drop('YearRemodAdd', axis =1, inplace = True)
fillvals = {'LotFrontage': 0, 'MasVnrArea': 0, 'GarageYrBlt': min(Housedat['GarageYrBlt'])}
Housedat.fillna(value = fillvals, inplace = True)
Housedat['GarageAge']=2010 - Housedat['GarageYrBlt']
Housedat.drop('GarageYrBlt', axis =1, inplace = True)
Housedat['MonthsSold']= 12*(2010 - Housedat['YrSold'])+(12-Housedat['MoSold'])
Housedat.drop(['YrSold', 'MoSold'], axis = 1, inplace = True)

In [10]:
for col in Housedat.columns:
    if np.max(Housedat[col].value_counts()) >= len(Housedat) * 0.9:
        Housedat = Housedat.drop(col, axis=1)

In [11]:
Housedat.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,GarageFinish_RFn,GarageFinish_Unf,GarageQual_TA,Fence_MnPrv,SaleType_WD,SaleCondition_Normal,HouseAge,RemodAge,GarageAge,MonthsSold
0,60,65.0,8450,7,5,196.0,706,0,150,856,...,1,0,1,0,1,1,7,7,7.0,34
1,20,80.0,9600,6,8,0.0,978,0,284,1262,...,1,0,1,0,1,1,34,34,34.0,43
2,60,68.0,11250,7,5,162.0,486,0,434,920,...,1,0,1,0,1,1,9,8,9.0,27
3,70,60.0,9550,7,5,0.0,216,0,540,756,...,0,1,1,0,1,0,95,40,12.0,58
4,60,84.0,14260,8,5,350.0,655,0,490,1145,...,1,0,1,0,1,1,10,10,10.0,24


In [12]:
Housedat = mean_normal(Housedat)

In [13]:
Housedat.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,GarageFinish_RFn,GarageFinish_Unf,GarageQual_TA,Fence_MnPrv,SaleType_WD,SaleCondition_Normal,HouseAge,RemodAge,GarageAge,MonthsSold
0,0.073350,0.212804,-0.207071,0.651256,-0.517023,0.513928,0.575228,-0.288554,-0.944267,-0.459145,...,1.567811,-0.840903,0.33701,-0.346999,0.390159,0.467491,-1.050634,-0.878367,-0.962220,0.133845
1,-0.872264,0.645526,-0.091855,-0.071812,2.178881,-0.570555,1.171591,-0.288554,-0.641008,0.466305,...,1.567811,-0.840903,0.33701,-0.346999,0.390159,0.467491,-0.156680,0.429430,-0.061681,0.704490
2,0.073350,0.299349,0.073455,0.651256,-0.517023,0.325803,0.092875,-0.288554,-0.301540,-0.313261,...,1.567811,-0.840903,0.33701,-0.346999,0.390159,0.467491,-0.984415,-0.829930,-0.895513,-0.309990
3,0.309753,0.068564,-0.096864,0.651256,-0.517023,-0.570555,-0.499103,-0.288554,-0.061648,-0.687089,...,-0.637395,1.188383,0.33701,-0.346999,0.390159,-2.137613,1.862993,0.720051,-0.795453,1.655565
4,0.073350,0.760919,0.375020,1.374324,-0.517023,1.366021,0.463410,-0.288554,-0.174805,0.199611,...,1.567811,-0.840903,0.33701,-0.346999,0.390159,0.467491,-0.951306,-0.733056,-0.862160,-0.500205


In [14]:
pmatrix = mylearn.pca(np.array(Housedat), 0.3)
pHouse = np.dot(Housedat, pmatrix)

In [15]:
pd.DataFrame(pHouse)

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,0.313395,-0.377397,0.241404,-0.769756,-0.286887,-1.475945,0.717252,-0.820513,-1.192110,-0.321173,...,-0.144033,0.609384,3.419795,-0.372945,0.255064,0.248201,2.387182,1.190587,1.922407,4.769471
1,-1.090010,2.066523,1.830474,1.540622,1.525867,0.809522,0.879188,1.964140,-1.249213,0.920577,...,1.169206,-0.142681,0.770058,3.123881,-0.596589,-0.072112,0.455117,0.240427,-2.075817,-0.756473
2,-0.393533,-0.801598,1.107256,-0.592018,-0.588127,-0.773631,1.446839,-1.240984,-0.744941,-1.201260,...,0.431171,0.879436,1.674432,0.383814,-0.784574,1.252686,2.791664,0.703302,1.106844,5.424367
3,1.693295,0.394394,1.082876,-2.251135,2.497427,0.321744,-1.537028,-0.838597,-1.122094,-0.462363,...,-3.074600,-0.292128,-2.112297,0.956647,-2.460402,0.854529,-1.219894,-2.586764,1.327039,-1.641325
4,-0.092535,-0.470908,0.927614,0.349671,-0.060203,-0.136923,1.657865,-1.569669,-1.678557,-0.844617,...,-0.468763,0.273449,0.597049,0.400890,-1.063544,0.633923,2.272839,-1.720145,0.482482,6.824560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,-0.378665,-0.325971,1.299123,0.850540,0.535304,0.273971,0.181080,2.077498,0.543449,-1.309312,...,0.957966,-0.948218,1.642771,-0.180802,1.509119,2.176856,0.959936,-0.076896,2.881659,1.962347
1456,-2.398267,1.225914,0.324687,0.736199,-1.534772,2.017582,1.085849,3.003795,0.189825,0.942618,...,-0.221644,-1.576655,1.397250,-0.544659,-0.472997,0.222531,0.307354,-1.438246,-3.975668,-0.294975
1457,1.421241,2.001948,-1.528792,-0.270315,-0.674846,-0.375321,-0.425167,-0.856299,-0.044257,3.485354,...,1.853848,0.989952,2.298225,0.024642,-1.516201,0.461366,-0.683323,-2.668987,1.983723,0.782709
1458,0.330077,-2.463311,-0.076815,0.372168,2.348578,-0.276901,-0.137377,0.200684,0.704655,2.809865,...,0.418024,-0.955719,3.112968,2.004519,-1.751910,-1.377424,-0.366444,1.455852,-3.559575,-3.271736


In [16]:
###create 4 folds do 4 fold k-folding

In [17]:
nfolds = 4
folds = np.random.choice(len(House), size = len(House), replace = False)
sfold = int(len(folds)/nfolds)

In [18]:
fold = []
for i in range(sfold):
    fold.append(folds[i*sfold:(i+1)*sfold])

In [19]:
Houseval = Housedat.loc[list(fold[3])]
Housetrain = Housedat.drop(fold[3])
valtarget = targets[fold[3]]
traintarget = targets.drop(fold[3])

In [20]:
probs = []
for val in np.unique(traintarget):
    prob = sum(traintarget == val)/len(traintarget)
    probs.append(prob)

In [21]:
probs

[0.08036529680365297,
 0.3415525114155251,
 0.28858447488584477,
 0.13607305936073058,
 0.15342465753424658]

In [22]:
mpptrain =  mylearn.mpp(np.array(Housetrain), np.array(traintarget), np.array(Housetrain), 2, probs, output=False)
mppval = mylearn.mpp(np.array(Housetrain), np.array(traintarget), np.array(Houseval), 2, probs, output=False)

In [23]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix
# Number of neurons chosen so number of weights is approximately 1095 / 10
net = MLPClassifier(hidden_layer_sizes=(7), activation='logistic', max_iter=5000)
net.fit(np.array(Housetrain), np.array(traintarget))

MLPClassifier(activation='logistic', hidden_layer_sizes=7, max_iter=5000)

In [24]:
bpnntrain = net.predict(np.array(Housetrain))
bpnnval = net.predict(np.array(Houseval))

In [25]:
from sklearn import svm
svm_clf = svm.SVC()
svm_clf.fit(Housetrain, traintarget)

SVC()

In [26]:
svmtrain = svm_clf.predict(np.array(Housetrain))
svmval = svm_clf.predict(np.array(Houseval))

In [27]:
len(svmval)

365

In [28]:
mppdec = np.zeros((5,5))

In [29]:
for idx, val in enumerate(traintarget):
    mppdec[val, int(mpptrain[idx])]+=1

In [30]:
mppdec

array([[ 70.,  17.,   1.,   0.,   0.],
       [ 23., 310.,  39.,   0.,   2.],
       [  0.,  60., 206.,  48.,   2.],
       [  0.,   1.,  28., 113.,   7.],
       [  0.,   0.,   6.,  25., 137.]])

In [31]:
bpnndec = np.zeros((5,5))

In [32]:
for idx, val in enumerate(traintarget):
    bpnndec[val, int(bpnntrain[idx])]+=1

In [33]:
bpnndec

array([[ 59.,  29.,   0.,   0.,   0.],
       [  6., 361.,   5.,   2.,   0.],
       [  0.,   4., 305.,   3.,   4.],
       [  0.,   0.,   1., 144.,   4.],
       [  0.,   0.,   0.,   6., 162.]])

In [34]:
svmdec = np.zeros((5,5))

In [35]:
for idx, val in enumerate(traintarget):
    svmdec[val, int(svmtrain[idx])]+=1

In [36]:
svmdec

array([[ 69.,  18.,   1.,   0.,   0.],
       [  0., 363.,  10.,   0.,   1.],
       [  0.,  20., 291.,   5.,   0.],
       [  0.,   1.,  16., 130.,   2.],
       [  0.,   0.,   3.,   6., 159.]])

In [37]:
for i in range(5):
    svmdec[i, :]/=svmdec[i, :].sum()
    bpnndec[i,:]/=bpnndec[i,:].sum()
    mppdec[i, :]/=mppdec[i, :].sum()

In [38]:
svmdec

array([[0.78409091, 0.20454545, 0.01136364, 0.        , 0.        ],
       [0.        , 0.97058824, 0.02673797, 0.        , 0.0026738 ],
       [0.        , 0.06329114, 0.92088608, 0.01582278, 0.        ],
       [0.        , 0.00671141, 0.10738255, 0.87248322, 0.01342282],
       [0.        , 0.        , 0.01785714, 0.03571429, 0.94642857]])

In [39]:
bpnndec

array([[0.67045455, 0.32954545, 0.        , 0.        , 0.        ],
       [0.01604278, 0.96524064, 0.01336898, 0.00534759, 0.        ],
       [0.        , 0.01265823, 0.96518987, 0.00949367, 0.01265823],
       [0.        , 0.        , 0.00671141, 0.96644295, 0.02684564],
       [0.        , 0.        , 0.        , 0.03571429, 0.96428571]])

In [40]:
vals = []
for i in range(len(valtarget)):
    mp = int(mppval[i])
    sv = int(svmval[i])
    bp = int(bpnnval[i])
    guess = np.argmax(mppdec[:,mp]*svmdec[:,sv]*bpnndec[:,bp])
    vals.append(guess)

In [41]:
NBClassAcc, NBOvAcc = util.accuracy_score(np.array(valtarget), vals) 

In [42]:
NBOvAcc

0.673972602739726

In [43]:
from scipy import stats

In [44]:
bksvals = []
for i in range(len(valtarget)):
    mp = int(mppval[i])
    sv = int(svmval[i])
    bp = int(bpnnval[i])
    combined = traintarget[(mpptrain==mp)&(svmtrain==sv)&(bpnntrain==bp)]
    if len(combined) == 0: 
        guess = bp
    else:
        guess = stats.mode(combined)[0][0]
    bksvals.append(guess)

In [45]:
BKSClassAcc, BKSOvAcc = util.accuracy_score(np.array(valtarget), bksvals) 

In [46]:
BKSOvAcc

0.6684931506849315